In [6]:
#!/bin/python

import sys
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics
from numpy import interp
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import plot_precision_recall_curve
from sklearn import preprocessing
from pathlib import Path

In [2]:
prediction_variable = 'ACR20'

list_n_tree = [10,30,100,300,1000]
list_n_max_depth = [2,3,4,5,7,10, None]
list_n_min_samples_split = [2,3,4,5]
list_n_min_samples_leaf = [1,2,3,4,5]
list_iteration = [1,2,3]

std_cutoff = 0.05
Path("./Hyperparameter").mkdir(parents=True, exist_ok=True)
Path("./Hyperparameter/RF").mkdir(parents=True, exist_ok=True)
output_file = './Hyperparameter/RF/performance_of_RF_models.txt'

df=pd.read_csv('./Input_data/RA_input.txt', sep='\t')

df=df[df.ACR20 != 3]

colnames = df.columns
result_column_list = ['newID', 'region', 'BSD', 'DFT1', 'DFT2', 'ACR20', 'ACR50', 'ACR70', 'EULAR']
x_colnames_1 = [item for item in colnames if item not in result_column_list] 
# They are not needed because it means results

In [ ]:
df_training = df[(df.region!=1) & (df.region != 11)]
df_independent = df[(df.region == 1) | (df.region == 11)]

In [3]:
# 1) Training dataset

df_training_remov_result = df_training[x_colnames_1] # pre-scaled data
# Remove variables includes only one value.
df_training_remov_novar = df_training_remov_result.loc[:,df_training_remov_result.std() != 0]

# Remove variables includes only small variance.
pre_scaler = preprocessing.MinMaxScaler()
df_training_pre_scaled = pre_scaler.fit_transform(df_training_remov_novar)
remain_boolean = df_training_pre_scaled.std(axis=0) >= std_cutoff
colnames_remain = df_training_remov_novar.columns[remain_boolean]
# colnames_remain will be used for independent data again.

data_x_training_bf_scaling = df_training_remov_novar[colnames_remain].to_numpy()
data_y_training = df_training[prediction_variable].to_numpy()

In [ ]:
# 2) Independent dataset

data_x_independent_bf_scaling = df_independent[colnames_remain].to_numpy()
data_y_independent = df_independent[prediction_variable].to_numpy()

In [ ]:
print(output_file)
perf_file = open(output_file, 'w')

for n_tree in list_n_tree:
    for n_max_depth in list_n_max_depth:
        for n_min_samples_split in list_n_min_samples_split:
            for n_min_samples_leaf in list_n_min_samples_leaf:
                for iteration in list_iteration:
                    
                    print(' '.join([prediction_variable, str(n_tree), str(n_max_depth), str(n_min_samples_split), str(n_min_samples_leaf), str(iteration)]))
                        
                    seed=random.randint(1,1000)
                    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

                    tprs = []
                    mean_fpr = np.linspace(0, 1, 100)
                    accs = []
                    ROC_aucs = []
                    f1s = []
                    RP_aucs = []

                    tprs_ind = []
                    accs_ind = []
                    ROC_aucs_ind = []
                    f1s_ind = []
                    RP_aucs_ind = []

                    tprs_ind_tscaler = []
                    accs_ind_tscaler = []
                    ROC_aucs_ind_tscaler = []
                    f1s_ind_tscaler = []
                    RP_aucs_ind_tscaler = []

                    i = 0

                    for train, test in cv.split(data_x_training_bf_scaling, data_y_training):

                        training_scaler = preprocessing.MinMaxScaler()

                        data_x_training_train = training_scaler.fit_transform(data_x_training_bf_scaling[train])
                        data_x_training_test = training_scaler.transform(data_x_training_bf_scaling[test])

                        model = RandomForestClassifier(n_estimators=n_tree, max_depth = n_max_depth, 
                                                       min_samples_split = n_min_samples_split, min_samples_leaf = n_min_samples_leaf)
                        model.fit(data_x_training_train, data_y_training[train])

                        test_acc = model.score(data_x_training_test, data_y_training[test])
                        predictions = model.predict_proba(data_x_training_test)[:,1]

                        accs.append(test_acc)

                        fpr, tpr, threshold = metrics.roc_curve(data_y_training[test], predictions)
                        roc_auc = metrics.auc(fpr, tpr)

                        tprs.append(interp(mean_fpr, fpr, tpr))
                        tprs[-1][0] = 0.0
                        ROC_aucs.append(roc_auc)

                        precision, recall, thresholds = precision_recall_curve(data_y_training[test], predictions)
                        f1 = f1_score(data_y_training[test], predictions.round())
                        f1s.append(f1)
                        rp_auc = metrics.auc(recall, precision)
                        RP_aucs.append(rp_auc)

                        # 1st method: independent dataset -> independent scaler

                        ind_scaler = preprocessing.MinMaxScaler()

                        data_x_independent = ind_scaler.fit_transform(data_x_independent_bf_scaling)

                        test_acc_ind = model.score(data_x_independent, data_y_independent)
                        predictions_ind = model.predict_proba(data_x_independent)[:,1]

                        accs_ind.append(test_acc_ind)

                        fpr_ind, tpr_ind, threshold = metrics.roc_curve(data_y_independent, predictions_ind)
                        roc_auc_ind = metrics.auc(fpr_ind, tpr_ind)

                        tprs_ind.append(interp(mean_fpr, fpr_ind, tpr_ind))
                        tprs_ind[-1][0] = 0.0
                        ROC_aucs_ind.append(roc_auc_ind)

                        precision_ind, recall_ind, thresholds = precision_recall_curve(data_y_independent, predictions_ind)
                        f1_ind = f1_score(data_y_independent, predictions_ind.round())
                        f1s_ind.append(f1_ind)
                        rp_auc_ind = metrics.auc(recall_ind, precision_ind)
                        RP_aucs_ind.append(rp_auc_ind)

                        # 2nd method: independent dataset scaling with training scaler

                        data_x_independent_tscaler = training_scaler.transform(data_x_independent_bf_scaling)

                        test_acc_ind_tscaler = model.score(data_x_independent_tscaler, data_y_independent)
                        predictions_ind_tscaler = model.predict_proba(data_x_independent_tscaler)[:,1]

                        accs_ind_tscaler.append(test_acc_ind_tscaler)

                        fpr_ind_tscaler, tpr_ind_tscaler, threshold = metrics.roc_curve(data_y_independent, predictions_ind_tscaler)
                        roc_auc_ind_tscaler = metrics.auc(fpr_ind_tscaler, tpr_ind_tscaler)

                        tprs_ind_tscaler.append(interp(mean_fpr, fpr_ind_tscaler, tpr_ind_tscaler))
                        tprs_ind_tscaler[-1][0] = 0.0
                        ROC_aucs_ind_tscaler.append(roc_auc_ind_tscaler)

                        precision_ind_tscaler, recall_ind_tscaler, thresholds = precision_recall_curve(data_y_independent, predictions_ind_tscaler)
                        f1_ind_tscaler = f1_score(data_y_independent, predictions_ind_tscaler.round())
                        f1s_ind_tscaler.append(f1_ind_tscaler)
                        rp_auc_ind_tscaler = metrics.auc(recall_ind_tscaler, precision_ind_tscaler)
                        RP_aucs_ind_tscaler.append(rp_auc_ind_tscaler)

                        i = i + 1

                    mean_tpr = np.mean(tprs, axis=0)
                    mean_tpr[-1] = 1.0
                    mean_auc = metrics.auc(mean_fpr, mean_tpr)
                    std_auc = np.std(ROC_aucs)

                    mean_tpr_ind = np.mean(tprs_ind, axis=0)
                    mean_tpr_ind[-1] = 1.0
                    mean_auc_ind = metrics.auc(mean_fpr, mean_tpr_ind)
                    std_auc_ind = np.std(ROC_aucs_ind)

                    mean_tpr_ind_tscaler = np.mean(tprs_ind_tscaler, axis=0)
                    mean_tpr_ind_tscaler[-1] = 1.0
                    mean_auc_ind_tscaler = metrics.auc(mean_fpr, mean_tpr_ind_tscaler)
                    std_auc_ind_tscaler = np.std(ROC_aucs_ind_tscaler)

                    perf_file.write('\t'.join([str(seed), str(n_tree), str(n_max_depth), str(n_min_samples_split), 
                                              str(n_min_samples_leaf), str(iteration), prediction_variable]) + '\t' + 
                                    '\t'.join([str(sum(accs) / 3), '\t'.join(str(x) for x in accs), 
                                               str(mean_auc), '\t'.join(str(x) for x in ROC_aucs),
                                               str(sum(f1s)/3), '\t'.join(str(x) for x in f1s),
                                               str(sum(RP_aucs) / 3), '\t'.join(str(x) for x in RP_aucs),
                                               str(sum(accs_ind) / 3), '\t'.join(str(x) for x in accs_ind), 
                                               str(mean_auc_ind), '\t'.join(str(x) for x in ROC_aucs_ind),
                                               str(sum(f1s_ind)/3), '\t'.join(str(x) for x in f1s_ind),
                                               str(sum(RP_aucs_ind) / 3), '\t'.join(str(x) for x in RP_aucs_ind),
                                               str(sum(accs_ind_tscaler) / 3), '\t'.join(str(x) for x in accs_ind_tscaler), 
                                               str(mean_auc_ind_tscaler), '\t'.join(str(x) for x in ROC_aucs_ind_tscaler),
                                               str(sum(f1s_ind_tscaler)/3), '\t'.join(str(x) for x in f1s_ind_tscaler),
                                               str(sum(RP_aucs_ind_tscaler) / 3), '\t'.join(str(x) for x in RP_aucs_ind_tscaler)]) + '\n')
                
perf_file.close()